In [67]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
import re
import pickle

In [55]:
def scrape_fa(year):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    my_headers = {
        'authority': 'en.wikipedia.org',
        'cache-control': 'max-age=0',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
        'cookie': 'WMF-Last-Access=09-Dec-2021; WMF-Last-Access-Global=09-Dec-2021; GeoIP=US:::37.75:-97.82:v4; enwikimwuser-sessionId=c57e79277dd10e90044c; enwikiel-sessionId=361cd58612199630ad70; enwikiwmE-sessionTickLastTickTime=1639063998485; enwikiwmE-sessionTickTickCount=10',
    }
    
    url = 'https://en.wikipedia.org/wiki/List_of_{0}%E2%80%93{1}_NBA_season_transactions'.format(year, str(year+1)[2:])

    response = session.get(url, headers=my_headers)
    print(response)
    soup = BeautifulSoup(response.text)
    
    if year == 2006:
        for t in soup.find_all('table', {'class': 'wikitable sortable'}):
            ths = str(t.find_all('th'))
            if ('Date signed' in ths) and ('New team' in ths) and ('Former team' in ths):
                player_htmls = soup.find_all('table', {'class': 'wikitable sortable'})[0].find_all('tr')[1:]
    elif year == 2010:
        t = soup.find_all('table', {'class': 'wikitable'})[5]
        player_htmls = t.find_all('td', {'align': 'left'})
    elif year == 2009:
        t = soup.find_all('table', {'class': 'wikitable'})[5]
        player_htmls = t.find_all('tr')[3:]
    else:
        for t in soup.find_all('table', {'class': 'wikitable sortable'}):
            ths = str(t.find_all('th'))
            if ('Date signed' in ths) and ('New team' in ths) and ('Former team' in ths):
                player_htmls = t.find_all('td', {'align': 'left'})
    try:
        players = [str(player_html.find('a').text) for player_html in player_htmls if player_html.find('a')]
        df = pd.DataFrame({'Player': players})
        df['previous season'] = str(year-1) + "-" + str(year)[2:]
    except UnboundLocalError as err:
        print(url)
        return None

    return df

In [60]:
with Pool(12) as p:
    free_agents = p.map(scrape_fa, list(range(2005, 2021)))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]><Response [200]>

<Response [200]>
<Response [200]>


In [61]:
free_agents = pd.concat(free_agents, axis=0)

In [69]:
with open('free_agents.pickle', 'wb') as handle:
    pickle.dump(free_agents, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

my_headers = {
    'authority': 'en.wikipedia.org',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
    'cookie': 'WMF-Last-Access=09-Dec-2021; WMF-Last-Access-Global=09-Dec-2021; GeoIP=US:::37.75:-97.82:v4; enwikimwuser-sessionId=c57e79277dd10e90044c; enwikiel-sessionId=361cd58612199630ad70; enwikiwmE-sessionTickLastTickTime=1639063998485; enwikiwmE-sessionTickTickCount=10',
}

url = 'https://en.wikipedia.org/wiki/List_of_{0}%E2%80%93{1}_NBA_season_transactions'.format(2009, '10')

soup = BeautifulSoup(session.get(url, headers=my_headers).text)

In [34]:
url

'https://en.wikipedia.org/wiki/List_of_2009%E2%80%9310_NBA_season_transactions'

In [46]:
player_htmls = soup.find_all('table', {'class': 'wikitable'})[5].find_all('tr')[3:]

In [54]:
[p.find('a') for p in player_htmls]

[<a href="/wiki/Trevor_Ariza" title="Trevor Ariza">Trevor Ariza</a>,
 <a class="mw-redirect" href="/wiki/Ron_Artest" title="Ron Artest">Ron Artest</a>,
 <a href="/wiki/Rasheed_Wallace" title="Rasheed Wallace">Rasheed Wallace</a>,
 <a class="mw-redirect" href="/wiki/Ben_Gordon_(basketball)" title="Ben Gordon (basketball)">Ben Gordon</a>,
 <a href="/wiki/Malik_Hairston" title="Malik Hairston">Malik Hairston</a>,
 <a href="/wiki/Marcus_Haislip" title="Marcus Haislip">Marcus Haislip</a>,
 <a href="/wiki/Quinton_Ross" title="Quinton Ross">Quinton Ross</a>,
 <a href="/wiki/Charlie_Villanueva" title="Charlie Villanueva">Charlie Villanueva</a>,
 <a href="/wiki/Chris_Andersen" title="Chris Andersen">Chris Andersen</a>,
 <a href="/wiki/Shannon_Brown" title="Shannon Brown">Shannon Brown</a>,
 <a href="/wiki/Anderson_Varej%C3%A3o" title="Anderson Varejão">Anderson Varejão</a>,
 <a href="/wiki/Brandon_Bass" title="Brandon Bass">Brandon Bass</a>,
 <a href="/wiki/Antonio_McDyess" title="Antonio McDye

In [53]:
None in [p.find('a') for p in player_htmls]

True

In [47]:
players = [str(player_html.find('a').text) for player_html in player_htmls]

AttributeError: 'NoneType' object has no attribute 'text'

In [88]:
for t in soup.find_all('table', {'class': 'wikitable sortable'}):
    ths = str(t.find_all('th'))
    print(ths)
    if ('Date signed' in ths) and ('New team' in ths) and ('Former team' in ths):
        test = t

[<th style="width:160px">Player
</th>, <th style="width:80px">Date signed
</th>, <th style="width:210px">New team
</th>, <th style="width:210px">Former team
</th>]
[<th style="width:30px">Pick
</th>, <th style="width:160px">Player
</th>, <th style="width:80px">Date signed
</th>, <th style="width:310px">Team signed
</th>, <th style="width:160px">School/Club Team
</th>]
[<th style="width:30px">Pick
</th>, <th style="width:160px">Player
</th>, <th style="width:80px">Date signed
</th>, <th style="width:310px">Team signed
</th>, <th style="width:160px">School/Club Team
</th>]
[<th>Date
</th>, <th>Player
</th>, <th>Team
</th>, <th>School/Club Team
</th>]
[<th style="width:160px">Player
</th>, <th style="width:80px">Date waived
</th>, <th style="width:180px">Former team
</th>]


In [94]:
test.find_all('td')

[<td><a href="/wiki/Mike_Gansey" title="Mike Gansey">Mike Gansey</a>
 </td>,
 <td><span data-sort-value="000000002021-07-05-0000" style="white-space:nowrap">July 5</span>
 </td>,
 <td><a href="/wiki/Miami_Heat" title="Miami Heat">Miami Heat</a>
 </td>,
 <td>West Virginia<br/><small>(went undrafted in 2006)</small>
 </td>,
 <td><a class="mw-redirect" href="/wiki/Louis_Amundson" title="Louis Amundson">Louis Amundson</a>
 </td>,
 <td><span data-sort-value="000000002021-07-06-0000" style="white-space:nowrap">July 6</span>
 </td>,
 <td><a href="/wiki/Sacramento_Kings" title="Sacramento Kings">Sacramento Kings</a>
 </td>,
 <td>UNLV<br/><small>(went undrafted in 2006)</small>
 </td>,
 <td><a href="/wiki/Sam_Cassell" title="Sam Cassell">Sam Cassell</a>
 </td>,
 <td rowspan="11"><span data-sort-value="000000002021-07-12-0000" style="white-space:nowrap">July 12</span>
 </td>,
 <td align="center" colspan="2"><a href="/wiki/Los_Angeles_Clippers" title="Los Angeles Clippers">Los Angeles Clippers</a